<a href="https://colab.research.google.com/github/AiNguyen2014/SoftwareEngineeringProject/blob/main/Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lưu ý

# Mình sẽ đánh giá essemble của train bằng silhoutte Và đánh giá external giữa independent với actual. Internal chỉ dùng để đánh giá train thôi

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')

TARGET_FOLDER = "Project Machine Learning"
BASE_PATH = None

for root, dirs, files in os.walk("/content/drive/MyDrive"):
    if TARGET_FOLDER in dirs:
        BASE_PATH = os.path.join(root, TARGET_FOLDER)
        break

if BASE_PATH is None:
    raise FileNotFoundError(" Không tìm thấy thư mục Project Machine Learning")

print(" Dùng thư mục:", BASE_PATH)
print(" File trong thư mục:", os.listdir(BASE_PATH))


ValueError: mount failed

# Setup môi trường

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
import warnings
from sklearn.decomposition import PCA
import joblib
warnings.filterwarnings('ignore')

# Tải dữ liệu

In [ ]:
# Train & Independent: gene là index
df_train = pd.read_csv(
    os.path.join(BASE_PATH, "data_set_ALL_AML_train.csv"),
    index_col=0
)

df_test = pd.read_csv(
    os.path.join(BASE_PATH, "data_set_ALL_AML_independent.csv"),
    index_col=0
)

# actual.csv: chỉ chứa nhãn → KHÔNG dùng index_col
df_actual = pd.read_csv(
    os.path.join(BASE_PATH, "actual.csv")
)

print("Train shape:", df_train.shape)
print("Independent shape:", df_test.shape)
print("Actual shape:", df_actual.shape)


# Tiền Xử Lý Dữ Liệu (Clean + Transpose + Z-score + PCA)

In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import f_classif
from sklearn.decomposition import PCA

# =============================
# CLEAN GENE EXPRESSION
# =============================
def clean_gene_expression(df):
    drop_cols = [
        col for col in df.columns
        if "Gene Description" in col
        or "Gene Accession Number" in col
        or "call" in col.lower()
    ]
    return df.drop(columns=drop_cols, errors="ignore")


df_train_clean = clean_gene_expression(df_train)
df_test_clean  = clean_gene_expression(df_test)

# =============================
# TRANSPOSE (samples × genes)
# =============================
X_train_raw = df_train_clean.T   # (38, genes)
X_test_raw  = df_test_clean.T    # (34, genes)

# =============================
# GỘP TOÀN BỘ 72 MẪU
# =============================
X_all_raw = pd.concat([X_train_raw, X_test_raw], axis=0)

print("Total samples after merge:", X_all_raw.shape)

# =============================
# Z-SCORE NORMALIZATION (ALL)
# =============================
scaler = StandardScaler()
X_all_scaled = scaler.fit_transform(X_all_raw)

print("Scaled shape:", X_all_scaled.shape)

# =============================
# FEATURE SELECTION (SUPERVISED – CHỈ ĐỂ GIẢM CHIỀU)
# =============================
N_GENES = 200

# Tính variance từng gene
gene_variances = np.var(X_all_scaled, axis=0)

# Chọn top N_GENES variance cao nhất
top_gene_idx = np.argsort(gene_variances)[-N_GENES:]
X_all_fs = X_all_scaled[:, top_gene_idx]

print(f"Selected top {N_GENES} genes (unsupervised)")
print("After FS shape:", X_all_fs.shape)

# =============================
# SAVE FINAL DATASET (KHÔNG PCA)
# =============================
# Lấy tên cột từ top gene index để dễ hiểu (nếu muốn)
gene_cols = [f"Gene_{i+1}" for i in top_gene_idx]

X_all_out = pd.DataFrame(
    X_all_fs,
    index=X_all_raw.index,
    columns=gene_cols
)
X_all_out.insert(0, "Sample_ID", X_all_out.index)

X_all_out.to_csv(
    os.path.join(BASE_PATH, "data_processed_72.csv"),
    index=False
)

print("\n=== PREPROCESSING COMPLETE (NO PCA) ===")
print("Saved to: data_processed_72.csv", X_all_fs.shape)

# BASE MODELS - K-MEANS++

## Import thư viện cần thiết

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score
import seaborn as sns
from scipy.spatial.distance import cdist
import warnings
warnings.filterwarnings('ignore')

# Thiết lập style cho biểu đồ
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

## Load dữ liệu đã chuẩn hóa

In [ ]:
# Đọc dữ liệu train đã được chuẩn hóa
try:
    train_path = os.path.join(BASE_PATH, 'data_processed_72.csv')
    train_df = pd.read_csv(train_path)
    print(f"Đã tải dữ liệu Train từ: {train_path}")
except Exception as e:
    print(f"Lỗi tải file: {e}")
    exit()

# Bỏ cột định danh không phải feature nếu tồn tại
if 'Sample_ID' in train_df.columns:
    train_df = train_df.drop(columns=['Sample_ID'])

# Chuyển thành numpy array
X_train = train_df.values  # hoặc train_df.to_numpy()


In [ ]:
# Chuyển đổi DataFrame sang numpy array để tính toán
X = train_df.values
print(f"\nMảng dữ liệu X có shape: {X.shape}")

## Định nghĩa các hàm cần dùng trong K means++

In [ ]:
# ═══ HÀM TÍNH KHOẢNG CÁCH ═══
def euclidean_distance(point1, point2):
    """Tính khoảng cách Euclidean giữa hai điểm"""
    return np.sqrt(np.sum((point1 - point2) ** 2))

def calculate_distances(X, centroids):
    """
    Tính ma trận khoảng cách từ TẤT CẢ điểm đến TẤT CẢ centroids
    Output: (n_samples, n_clusters)
    """
    n_samples = X.shape[0]
    n_clusters = centroids.shape[0]
    distances = np.zeros((n_samples, n_clusters))

    for i in range(n_samples):
        for j in range(n_clusters):
            distances[i, j] = euclidean_distance(X[i], centroids[j])

    return distances

# ═══ HÀM TÍNH SILHOUETTE SCORE (dùng sklearn) ═══
def calculate_silhouette_score(X, labels):
    """Tính Silhouette Score bằng sklearn.metrics.silhouette_score"""
    return silhouette_score(X, labels)
print("Đã định nghĩa hàm tiện ích: khoảng cách, silhouette score (sklearn)")


# ═══ BƯỚC 2: KHỞI TẠO CENTROIDS (K-MEANS++) ═══
def initialize_centroids_kmeans_plusplus(X, k, random_state=None):

    if random_state is not None:
        np.random.seed(random_state)

    n_samples = X.shape[0]
    centroids = [X[np.random.randint(0, n_samples)]]

    for _ in range(1, k):
        distances = np.array([min([euclidean_distance(x, c) for c in centroids]) for x in X])
        probabilities = distances ** 2
        probabilities /= probabilities.sum()
        next_centroid_idx = np.random.choice(n_samples, p=probabilities)
        centroids.append(X[next_centroid_idx])

    return np.array(centroids)

# ═══ BƯỚC 3: GÁN CỤM ═══
def assign_clusters(X, centroids):

    distances = calculate_distances(X, centroids)
    return np.argmin(distances, axis=1)

# ═══ BƯỚC 4: CẬP NHẬT CENTROIDS ═══
def update_centroids(X, labels, k):

    n_features = X.shape[1]
    centroids = np.zeros((k, n_features))

    for i in range(k):
        cluster_points = X[labels == i]
        if len(cluster_points) > 0:
            centroids[i] = cluster_points.mean(axis=0)
        else:
            centroids[i] = X[np.random.randint(0, X.shape[0])]

    return centroids

# ═══ HÀM K-MEANS++ HOÀN CHỈNH ═══
def kmeans_plusplus(X, k, max_iters=100, tol=1e-4, random_state=None, verbose=False):

    if random_state is not None:
        np.random.seed(random_state)

    # BƯỚC 2: Khởi tạo centroids
    centroids = initialize_centroids_kmeans_plusplus(X, k, random_state)

    history = {
        'iterations': 0,
        'shifts': [],
        'converged': False
    }

    for iteration in range(max_iters):
        # BƯỚC 3: Gán các điểm vào cụm
        labels = assign_clusters(X, centroids)

        # Lưu centroid cũ
        old_centroids = centroids.copy()

        # BƯỚC 4: Cập nhật centroids
        centroids = update_centroids(X, labels, k)

        # BƯỚC 5: Kiểm tra hội tụ
        centroid_shift = np.sum([euclidean_distance(old_centroids[i], centroids[i])
                                for i in range(k)])

        history['shifts'].append(centroid_shift)
        history['iterations'] = iteration + 1

        if verbose and (iteration + 1) % 10 == 0 or iteration == 0:
            print(f"  Iteration {iteration + 1}: Shift={centroid_shift:.2e}")

        if centroid_shift < tol:
            history['converged'] = True
            if verbose:
                print(f"  Hội tụ tại iteration {iteration + 1}")
            break

    return labels, centroids, history

print("Đã định nghĩa các hàm hỗ trợ (Bước 2, 3, 4, 5)")
print("  (Sẽ sử dụng ở BƯỚC 1: Đánh giá K, và BƯỚC 2-5: Huấn luyện)")



## BƯỚC 1: ĐÁNH GIÁ & LỰA CHỌN SỐ CỤM K TỐI ƯU
**Thực hiện:** Chạy K-means++ với K từ 2 đến 10, so sánh Silhouette Score, chọn K tốt nhất

In [ ]:
"""
═══════════════════════════════════════════════════════════════════════════════
 BƯỚC 1: ĐÁNH GIÁ - CHẠY K-MEANS++ VỚI K = 2 ĐẾN 10
═══════════════════════════════════════════════════════════════════════════════
MỤC TIÊU: Tìm K tối ưu bằng Silhouette Score
MỤC ĐÍCH: Xác định giá trị K tốt nhất trước khi huấn luyện chi tiết (Bước 2-5)
"""

print("\n" + "="*80)
print("BƯỚC 1: ĐÁNH GIÁ LỰA CHỌN SỐ CỤM K TỐI ƯU")
print("="*80)
print("\nChạy K-means++ với K từ 2 đến 10 trên dữ liệu train_scaled.csv")
print("So sánh Silhouette Score để lựa chọn K tốt nhất\n")

k_values = range(2, 11)
silhouette_scores = []

for k in k_values:
    print(f"[K={k:2d}]", end=" ")

    # Chạy K-MEANS++ với k cụm
    labels, centroids, history = kmeans_plusplus(X, k, max_iters=300, random_state=42)

    # Tính Silhouette Score
    silhouette = calculate_silhouette_score(X, labels)
    silhouette_scores.append(silhouette)

    print(f"Silhouette = {silhouette:7.4f} | Iterations = {history['iterations']}")

print("\n" + "-"*80)

# Tìm k tốt nhất
best_k_idx = np.argmax(silhouette_scores)
best_k = list(k_values)[best_k_idx]
best_silhouette = silhouette_scores[best_k_idx]

print(f"\nKẾT QUẢ ĐÁNH GIÁ:")
print(f"   K có Silhouette Score cao nhất: K = {best_k}")
print(f"   Silhouette Score: {best_silhouette:.4f}")
print(f"\n   SẼ CHỌN: K = {best_k} cho bước huấn luyện tiếp theo")
print(f"{'='*80}\n")


In [ ]:
"""
═══════════════════════════════════════════════════════════════════════════════
    BƯỚC 2-5: HUẤN LUYỆN MÔ HÌNH K-MEANS++ CHI TIẾT
═══════════════════════════════════════════════════════════════════════════════
MỤC TIÊU: Sử dụng K tốt nhất từ BƯỚC 1 để huấn luyện mô hình chi tiết
MỤC ĐÍCH: Theo dõi từng iteration - gán → cập nhật → kiểm tra hội tụ
"""

print("\n" + "="*80)
print("BƯỚC 2-5: HUẤN LUYỆN MÔ HÌNH K-MEANS++ CHI TIẾT")
print("="*80)
print(f"\nSỐ CỤM ĐƯỢC CHỌN: K = {best_k}")
print(f"  (Dựa trên Silhouette Score cao nhất = {best_silhouette:.4f})\n")

# Chạy K-means với K tốt nhất, verbose để thấy quá trình
print(f"{'─'*80}")
print("Chạy K-means++ với K = {} trên dữ liệu train_scaled.csv".format(best_k))
print(f"{'─'*80}\n")

labels_final, centroids_final, training_history = kmeans_plusplus(
    X, best_k, max_iters=100, tol=1e-4, random_state=42, verbose=True
)

print(f"\n{'─'*80}")
if training_history['converged']:
    print(f"THUẬT TOÁN HỘI TỤ sau {training_history['iterations']} iterations")
else:
    print(f"ĐẠT SỐ ITERATIONS TỐI ĐA ({training_history['iterations']})")
print(f"{'─'*80}\n")

# Lưu lịch sử
history_centroid_shifts = training_history['shifts']
iteration_count = training_history['iterations']


## ĐÁNH GIÁ KẾT QUẢ MÔ HÌNH TRÊN DỮ LIỆU TRAIN

In [ ]:
print("\n" + "="*80)
print("THỐNG KÊ CHI TIẾT VỀ KẾT QUẢ HỌC")
print("="*80)

# Tính chỉ số cuối cùng
final_silhouette = calculate_silhouette_score(X, labels_final)

print(f"\nTHÔNG TIN CHUNG:")
print(f"   - Dữ liệu: train_scaled.csv")
print(f"   - Số mẫu (samples): {X.shape[0]}")
print(f"   - Số features: {X.shape[1]}")
print(f"   - Số cụm (K): {best_k}")

print(f"\nCHỈ SỐ ĐÁNH GIÁ:")
print(f"   - Silhouette Score: {final_silhouette:.4f}")

print(f"\nTHÔNG TIN HỌC:")
print(f"   - Số iterations: {iteration_count}")
print(f"   - Trạng thái: {'Hội tụ' if training_history['converged'] else 'Max iterations'}")

print(f"\nPHÂN BỐ CÁC CỤM:")
for cluster_id in range(best_k):
    count = np.sum(labels_final == cluster_id)
    percentage = (count / len(labels_final)) * 100
    print(f"   - Cụm {cluster_id}: {count:4d} điểm ({percentage:5.1f}%)")

print(f"\nTHÔNG TIN CENTROIDS:")
for i, centroid in enumerate(centroids_final):
    print(f"   Centroid {i}:")
    print(f"     - Giá trị trung bình: {centroid.mean():.4f}")
    print(f"     - Độ lệch chuẩn: {centroid.std():.4f}")
    print(f"     - Range: [{centroid.min():.4f}, {centroid.max():.4f}]")

print(f"\n{'='*80}\n")


# BASE MODELS - HIERARCHICAL

# **Xây dựng mô hình (Training) và đánh giá nội bộ (chưa đụng đến tập Test hay so sánh với thực tế)**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import silhouette_score
import os

# ============================================================
# PHẦN 1: BASE LEARNER - HIERARCHICAL CLUSTERING
# ============================================================

class HierarchicalClustering:
    def __init__(self, n_clusters=2, linkage='ward'):
        self.n_clusters = n_clusters
        self.linkage = linkage
        self.labels_ = None

    def _euclidean_distance(self, point1, point2):
        return np.sqrt(np.sum((point1 - point2) ** 2))

    def _compute_distance_matrix(self, X):
        n = X.shape[0]
        dist_matrix = np.zeros((n, n))
        for i in range(n):
            for j in range(i+1, n):
                dist = self._euclidean_distance(X[i], X[j])
                dist_matrix[i, j] = dist
                dist_matrix[j, i] = dist
        return dist_matrix

    # --- Linkage Functions ---
    def _single_linkage(self, c1_idxs, c2_idxs, dist_matrix):
        min_dist = float('inf')
        for i in c1_idxs:
            for j in c2_idxs:
                if dist_matrix[i, j] < min_dist: min_dist = dist_matrix[i, j]
        return min_dist

    def _complete_linkage(self, c1_idxs, c2_idxs, dist_matrix):
        max_dist = 0
        for i in c1_idxs:
            for j in c2_idxs:
                if dist_matrix[i, j] > max_dist: max_dist = dist_matrix[i, j]
        return max_dist

    def _ward_linkage(self, c1_idxs, c2_idxs, X):
        m1 = np.mean(X[c1_idxs], axis=0)
        m2 = np.mean(X[c2_idxs], axis=0)
        n1, n2 = len(c1_idxs), len(c2_idxs)
        return np.sqrt((2 * n1 * n2) / (n1 + n2)) * self._euclidean_distance(m1, m2)

    def _cluster_distance(self, c1_idxs, c2_idxs, dist_matrix, X):
        if self.linkage == 'single': return self._single_linkage(c1_idxs, c2_idxs, dist_matrix)
        elif self.linkage == 'complete': return self._complete_linkage(c1_idxs, c2_idxs, dist_matrix)
        elif self.linkage == 'ward': return self._ward_linkage(c1_idxs, c2_idxs, X)

    def fit(self, X):
        n_samples = X.shape[0]
        clusters = {i: [i] for i in range(n_samples)}

        dist_matrix = None
        if self.linkage != 'ward':
            dist_matrix = self._compute_distance_matrix(X)

        while len(clusters) > self.n_clusters:
            min_dist = float('inf')
            merge_pair = None
            ids = list(clusters.keys())

            # Tìm cặp gần nhất
            for i in range(len(ids)):
                for j in range(i+1, len(ids)):
                    id1, id2 = ids[i], ids[j]
                    dist = self._cluster_distance(clusters[id1], clusters[id2], dist_matrix, X)
                    if dist < min_dist:
                        min_dist = dist
                        merge_pair = (id1, id2)

            # Gộp
            c1, c2 = merge_pair
            new_id = max(ids) + 1
            clusters[new_id] = clusters[c1] + clusters[c2]
            del clusters[c1]; del clusters[c2]

        # Lưu nhãn cuối cùng
        self.labels_ = np.zeros(n_samples, dtype=int)
        for idx, (cid, members) in enumerate(clusters.items()):
            self.labels_[members] = idx
        return self

# ============================================================
# PHẦN 2: CHUẨN BỊ CHO ENSEMBLE (BASE LEARNERS)
# ============================================================

if __name__ == "__main__":
    # 1. Load Data
    try:
        train_path = os.path.join(BASE_PATH, 'data_processed_72.csv')
        train_df = pd.read_csv(train_path)
        X_train = train_df.drop('Sample_ID', axis=1, errors='ignore').values
        print(f" Đã tải dữ liệu: {X_train.shape}")
    except Exception as e:
        print(f"Lỗi: {e}"); exit()

    # 2. Tạo tập hợp các Base Learners (Đa dạng hóa bằng Linkage)
    # Lưu ý quan trọng: n_clusters=2 (Chuẩn bài toán ung thư)
    linkages = ['single', 'complete', 'ward']
    base_models = {}

    print("\n ĐANG XÂY DỰNG CÁC BASE LEARNERS...")

    for link in linkages:
        print(f"   -> Training {link.upper()} model...")
        model = HierarchicalClustering(n_clusters=2, linkage=link)
        model.fit(X_train)

        # Lưu model vào dictionary để dùng cho Ensemble sau này
        base_models[link] = model

        # Đánh giá nhanh chất lượng từng model con
        sil = silhouette_score(X_train, model.labels_)
        print(f"      (Silhouette: {sil:.4f})")

    print(f"\n Đã chuẩn bị xong {len(base_models)} mô hình cơ sở cho Ensemble!")
    print(f"   Danh sách: {list(base_models.keys())}")

# BASE MODELS - GMM

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from sklearn.metrics import silhouette_score, davies_bouldin_score

# ============================================================
# LOAD DATA (SAU PCA)
# ============================================================

train_df = pd.read_csv(os.path.join(BASE_PATH, "data_processed_72.csv"))

X_train = train_df.drop(columns=["Sample_ID"], errors="ignore").values

print("Train shape:", X_train.shape)


# ============================================================
# GMM - TỰ CÀI ĐẶT (EM ALGORITHM)
# ============================================================

class GMM:
    def __init__(self, n_components=2, max_iter=100, tol=1e-4):
        self.K = n_components
        self.max_iter = max_iter
        self.tol = tol

    def gaussian(self, X, mean, cov):
        d = X.shape[1]
        cov_reg = cov + np.eye(d) * 1e-6   # regularization
        inv = np.linalg.inv(cov_reg)
        det = np.linalg.det(cov_reg)
        diff = X - mean
        expo = np.sum(diff @ inv * diff, axis=1)
        coef = 1.0 / np.sqrt((2 * np.pi) ** d * det)
        return coef * np.exp(-0.5 * expo)

    def fit(self, X):
        n, d = X.shape
        rng = np.random.default_rng(42)

        # Khởi tạo
        self.means = X[rng.choice(n, self.K, replace=False)]
        base_cov = np.cov(X.T)
        self.covs = np.array([base_cov.copy() for _ in range(self.K)])
        self.weights = np.ones(self.K) / self.K

        prev_ll = None

        for _ in range(self.max_iter):
            resp = np.zeros((n, self.K))

            # E-step
            for k in range(self.K):
                resp[:, k] = self.weights[k] * self.gaussian(
                    X, self.means[k], self.covs[k]
                )

            resp_sum = resp.sum(axis=1, keepdims=True)
            resp_sum[resp_sum == 0] = 1e-10
            resp /= resp_sum

            Nk = resp.sum(axis=0)

            # M-step
            for k in range(self.K):
                self.means[k] = np.sum(resp[:, k][:, None] * X, axis=0) / Nk[k]
                diff = X - self.means[k]
                self.covs[k] = (resp[:, k][:, None] * diff).T @ diff / Nk[k]
                self.weights[k] = Nk[k] / n

            ll = np.sum(np.log(resp_sum))
            if prev_ll is not None and abs(ll - prev_ll) < self.tol:
                break
            prev_ll = ll

    def predict(self, X):
        probs = np.zeros((X.shape[0], self.K))
        for k in range(self.K):
            probs[:, k] = self.weights[k] * self.gaussian(
                X, self.means[k], self.covs[k]
            )
        return np.argmax(probs, axis=1)


# ============================================================
# TRAIN GMM
# ============================================================

gmm = GMM(n_components=2)
gmm.fit(X_train)

train_labels = gmm.predict(X_train)
test_labels  = gmm.predict(X_test)

print("Train cluster labels:", train_labels)
print("Independent cluster labels:", test_labels)


# ============================================================
# INTERNAL VALIDATION (TRAIN ONLY)
# ============================================================

sil_score = silhouette_score(X_train, train_labels)
db_score  = davies_bouldin_score(X_train, train_labels)

print("\nINTERNAL VALIDATION (TRAIN - GMM)")
print("Silhouette score:", round(sil_score, 4))
print("Davies–Bouldin index:", round(db_score, 4))


# ============================================================
# VISUALIZATION (PCA 2D)
# ============================================================

X_vis = X_train[:, :2]

plt.figure(figsize=(6, 5))
plt.scatter(
    X_vis[:, 0], X_vis[:, 1],
    c=train_labels,
    cmap="viridis",
    s=60
)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title("GMM clustering on TRAIN (PCA space)")
plt.colorbar(label="Cluster")
plt.tight_layout()
plt.show()


# ============================================================
# OUTPUT FOR ENSEMBLE
# ============================================================

gmm_labels_train = train_labels
gmm_labels_test  = test_labels


# ENSEMBLE CLUSTERING

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

print("--- Đang lấy nhãn từ 3 Base Models ---")

# 1. Base models

labels_km, _, _ = kmeans_plusplus(X_all, k=2, random_state=42)

bu_model.fit(X_all)
labels_bu = bu_model.labels_

gmm_model.fit(X_all)
labels_gmm = gmm_model.predict(X_all)

score = silhouette_score(X_all, ensemble_labels)

print("Đã thu thập xong nhãn từ K-means++, Bottom-up và GMM.")

# 2. Weights
weights = {
    "kmeans": 0.3,
    "hierarchical": 0.6,
    "gmm": 0.1
}

labels_dict = {
    "kmeans": labels_km,
    "hierarchical": labels_bu,
    "gmm": labels_gmm
}

# ============================================================
# 3. BUILD WEIGHTED CO-ASSOCIATION MATRIX
# ============================================================
def build_weighted_co_association_matrix(labels_dict, weights):
    model_names = list(labels_dict.keys())
    n_samples = len(labels_dict[model_names[0]])

    co_matrix = np.zeros((n_samples, n_samples))
    total_weight = sum(weights.values())

    for name in model_names:
        labels = labels_dict[name]
        w = weights[name]

        for i in range(n_samples):
            for j in range(i, n_samples):
                if labels[i] == labels[j]:
                    co_matrix[i, j] += w
                    if i != j:
                        co_matrix[j, i] += w

    return co_matrix / total_weight


print("\n--- Đang xây dựng Weighted Co-association Matrix ---")
co_matrix = build_weighted_co_association_matrix(labels_dict, weights)

# ============================================================
# 4. FINAL ENSEMBLE CLUSTERING
# ============================================================
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import squareform

def ensemble_final_step(co_matrix, n_clusters=2):
    # Similarity → Distance
    dist_matrix = 1 - co_matrix

    condensed_dist = squareform(dist_matrix, checks=False)
    Z = linkage(condensed_dist, method='average')

    final_labels = fcluster(Z, t=n_clusters, criterion='maxclust') - 1
    return final_labels, Z


print("--- Đang thực hiện Final Hierarchical Clustering ---")
ensemble_labels, linkage_matrix = ensemble_final_step(co_matrix, n_clusters=2)

# ============================================================
# 5. EVALUATION
# ============================================================
from sklearn.metrics import silhouette_score

score = silhouette_score(X_train, ensemble_labels)

print("\n" + "="*50)
print("KẾT QUẢ ENSEMBLE CLUSTERING (KM++ + BU + GMM)")
print("="*50)
print(f"Final Silhouette Score: {score:.4f}")
print(f"Nhãn cụm cuối cùng: {ensemble_labels}")

# ============================================================
# 6. VISUALIZATION
# ============================================================
plt.figure(figsize=(8, 6))
sns.heatmap(co_matrix, cmap='YlGnBu')
plt.title("Weighted Co-association Matrix")
plt.show()

plt.figure(figsize=(10, 5))
from scipy.cluster.hierarchy import dendrogram
dendrogram(linkage_matrix)
plt.title("Ensemble Dendrogram")
plt.show()

# ============================================================
# 7. EXTERNAL EVALUATION
# ============================================================

df_actual = pd.read_csv(
    os.path.join(BASE_PATH, "actual.csv")
)

X_train_patient_ids = list(range(1, 39))  # nếu bệnh nhân train là 1-38

# Lọc nhãn train
train_actual = df_actual[df_actual['patient'].isin(X_train_patient_ids)]

# Map nhãn sang 0/1
y_true = train_actual['cancer'].map({'AML': 0, 'ALL': 1}).values

# Kiểm tra
print("Train y_true shape:", y_true.shape)
print(y_true)


from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score

def purity_score(y_true, y_pred):
    # Purity = sum(max count in cluster) / total samples
    contingency_matrix = np.zeros((np.max(y_pred)+1, np.max(y_true)+1))
    for i in range(len(y_true)):
        contingency_matrix[y_pred[i], y_true[i]] += 1
    return np.sum(np.max(contingency_matrix, axis=1)) / np.sum(contingency_matrix)

# y_true = nhãn thực tế ALL/AML từ actual.csv, dạng integer 0/1
# Giả sử mày đã encode nhãn thành 0 = AML, 1 = ALL

base_models = {
    "KMeans++": labels_km,
    "Hierarchical": labels_bu,
    "GMM": labels_gmm,
    "Ensemble": ensemble_labels
}

print("\n--- EXTERNAL EVALUATION (Train) ---")
for name, labels in base_models.items():
    ari = adjusted_rand_score(y_true, labels)
    nmi = normalized_mutual_info_score(y_true, labels)
    pur = purity_score(y_true, labels)
    print(f"{name}: ARI={ari:.4f}, NMI={nmi:.4f}, Purity={pur:.4f}")


#  TEST ON INDEPENDENT

In [ ]:
import pandas as pd
import numpy as np
import os

# Load test
df_test = pd.read_csv(os.path.join(BASE_PATH, "test_processed.csv"))
X_test = df_test.values
# Hoặc nếu có cột Sample_ID:
X_test = df_test.drop(columns=["Sample_ID"], errors="ignore").values



print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
X_all = np.vstack([X_train, X_test])
n_train = X_train.shape[0]
n_test = X_test.shape[0]

print("X_all shape:", X_all.shape)
print("\n--- BASE MODELS trên TRAIN + TEST ---")

# KMeans++
labels_km_all, _, _ = kmeans_plusplus(
    X_all,
    k=2,
    random_state=42
)

# Hierarchical
bu_all = HierarchicalClustering(n_clusters=2, linkage='ward')
bu_all.fit(X_all)
labels_bu_all = bu_all.labels_

# GMM
gmm_all = GMM(n_components=2)
gmm_all.fit(X_all)
labels_gmm_all = gmm_all.predict(X_all)

print("Đã lấy xong nhãn KMeans++, Hierarchical, GMM cho toàn bộ data.")
weights = {
    "kmeans": 0.3,
    "hierarchical": 0.6,
    "gmm": 0.1
}

labels_dict_all = {
    "kmeans": labels_km_all,
    "hierarchical": labels_bu_all,
    "gmm": labels_gmm_all
}

def build_weighted_co_association_matrix(labels_dict, weights):
    model_names = list(labels_dict.keys())
    n_samples = len(labels_dict[model_names[0]])

    co_matrix = np.zeros((n_samples, n_samples))
    total_weight = sum(weights.values())

    for name in model_names:
        labels = labels_dict[name]
        w = weights[name]

        for i in range(n_samples):
            for j in range(i, n_samples):
                if labels[i] == labels[j]:
                    co_matrix[i, j] += w
                    if i != j:
                        co_matrix[j, i] += w

    return co_matrix / total_weight


print("\n--- Xây dựng EXTENDED Co-association Matrix ---")
co_matrix_all = build_weighted_co_association_matrix(labels_dict_all, weights)

print("Co-association shape:", co_matrix_all.shape)
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import squareform

def ensemble_final_step(co_matrix, n_clusters=2):
    dist_matrix = 1 - co_matrix
    condensed_dist = squareform(dist_matrix, checks=False)
    Z = linkage(condensed_dist, method='average')
    final_labels = fcluster(Z, t=n_clusters, criterion='maxclust') - 1
    return final_labels, Z


print("\n--- CONSENSUS CLUSTERING trên EXTENDED MATRIX ---")
ensemble_labels_all, linkage_matrix_all = ensemble_final_step(
    co_matrix_all,
    n_clusters=2
)

labels_train_final = ensemble_labels_all[:n_train]
labels_test_final  = ensemble_labels_all[n_train:]

print("Train ensemble labels shape:", labels_train_final.shape)
print("Test ensemble labels shape:", labels_test_final.shape)
df_actual = pd.read_csv(os.path.join(BASE_PATH, "actual.csv"))

# Train: patient 1–38
train_actual = df_actual[df_actual['patient'].between(1, 38)]
y_train_true = train_actual['cancer'].map({'AML': 0, 'ALL': 1}).values

# Test: patient 39–72
test_actual = df_actual[df_actual['patient'].between(39, 72)]
y_test_true = test_actual['cancer'].map({'AML': 0, 'ALL': 1}).values
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score

def purity_score(y_true, y_pred):
    contingency = np.zeros((np.max(y_pred)+1, np.max(y_true)+1))
    for i in range(len(y_true)):
        contingency[y_pred[i], y_true[i]] += 1
    return np.sum(np.max(contingency, axis=1)) / np.sum(contingency)


print("\n--- EXTERNAL EVALUATION (EXTENDED ENSEMBLE) ---")

print("TRAIN:")
print("ARI:", adjusted_rand_score(y_train_true, labels_train_final))
print("NMI:", normalized_mutual_info_score(y_train_true, labels_train_final))
print("Purity:", purity_score(y_train_true, labels_train_final))

print("\nTEST:")
print("ARI:", adjusted_rand_score(y_test_true, labels_test_final))
print("NMI:", normalized_mutual_info_score(y_test_true, labels_test_final))
print("Purity:", purity_score(y_test_true, labels_test_final))


#  EVALUATION